In [1]:
import sys

sys.path.append("../..")
from tools.task_description import display_description

display_description()

# Sub-string Divisibility

[Problem Link](https://projecteuler.net/problem=43)

The number, $1406357289$, is a $0$ to $9$ pandigital number because it is made up of each of the digits $0$ to $9$ in some order, but it also has a rather interesting sub-string divisibility property.

Let $d_1$ be the $1$st digit, $d_2$ be the $2$nd digit, and so on. In this way, we note the following:

* $d_2d_3d_4=406$ is divisible by $2$
* $d_3d_4d_5=063$ is divisible by $3$
* $d_4d_5d_6=635$ is divisible by $5$
* $d_5d_6d_7=357$ is divisible by $7$
* $d_6d_7d_8=572$ is divisible by $11$
* $d_7d_8d_9=728$ is divisible by $13$
* $d_8d_9d_{10}=289$ is divisible by $17$

Find the sum of all $0$ to $9$ pandigital numbers with this property.

## Brute-force Solution

In [19]:
from itertools import permutations


def main_bf():
    total_sum = 0
    for perm in permutations(range(10), 10):
        d1, d2, d3, d4, d5, d6, d7, d8, d9, d10 = perm
        if d4 % 2 != 0:
            continue
        if (d3 + d4 + d5) % 3 != 0:
            continue
        if d6 not in (0, 5):
            continue
        if (100 * d5 + 10 * d6 + d7) % 7 != 0:
            continue
        if (100 * d6 + 10 * d7 + d8) % 11 != 0:
            continue
        if (100 * d7 + 10 * d8 + d9) % 13 != 0:
            continue
        if (100 * d8 + 10 * d9 + d10) % 17 != 0:
            continue
        number = (
            d1 * 10**9
            + d2 * 10**8
            + d3 * 10**7
            + d4 * 10**6
            + d5 * 10**5
            + d6 * 10**4
            + d7 * 10**3
            + d8 * 10**2
            + d9 * 10**1
            + d10
        )
        total_sum += number
    return total_sum


In [24]:
%%timeit
main_bf()

398 ms ± 7.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
main_bf()

16695334890

## Optimized Solution
- notes  

In [29]:
def main_opt():
    # Divisors for sub-strings starting at d2, d3, ..., d8
    # d2d3d4 % 2 == 0 ... d8d9d10 % 17 == 0
    divisors = [2, 3, 5, 7, 11, 13, 17]

    def search(current_suffix, divisor_idx):
        # Base case: We have satisfied all divisors (divisor_idx goes 5 -> -1)
        # current_suffix contains d2...d10
        if divisor_idx < 0:
            # Find the single missing digit for d1
            missing = set("0123456789") - set(current_suffix)
            d1 = missing.pop()
            return int(d1 + current_suffix)

        total = 0
        target_divisor = divisors[divisor_idx]

        # The new digit 'd' will be placed at the front.
        # The sub-string to check is d + current_suffix[:2]
        # e.g., if current is d8d9d10, we add d7, check d7d8d9 against 13
        suffix_head = current_suffix[:2]

        for d in "0123456789":
            if d not in current_suffix:
                if int(d + suffix_head) % target_divisor == 0:
                    total += search(d + current_suffix, divisor_idx - 1)
        return total

    total_sum = 0
    # Start from the end: d8d9d10 must be divisible by 17
    # Multiples of 17 are much sparser than multiples of 2, reducing search space immediately.
    for i in range(17, 1000, 17):
        s = f"{i:03d}"
        if len(set(s)) == 3:
            # We have a valid d8d9d10.
            # Now search for d7 using divisors[5] (which is 13)
            total_sum += search(s, 5)

    return total_sum

In [30]:
%%timeit
main_opt()

139 μs ± 2.16 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
main_opt()

16695334890